In [1]:
import requests
import re
import pandas as pd
import datetime
import json

In [10]:
multiplier_path = "/Users/kai/repositories/ynab-importer/multiplier_jul2020_test.csv"
livefresh_path = "/Users/kai/repositories/ynab-importer/LiveFresh current - Sheet1.csv"

In [31]:
def truncate_file(fpath, pattern):
    """Locate the line where the table starts and truncate the file"""
    with open(fpath, 'r') as fopen:
        lines = fopen.readlines()
        for i in range(len(lines)):
            if re.match(pattern, lines[i]):
                break
        res = [l.strip() for l in lines[i:] if l != '\n']
    with open('temp.csv', 'w') as fout:
        for l in res:
            fout.write(l[:-1]+'\n')
    return res

In [32]:
truncated = truncate_file(multiplier_path, 'Transaction Date')

In [33]:
df = pd.read_csv('temp.csv')

In [34]:
df.head()

,Transaction Date,Value Date,Statement Code,Reference,Debit Amount,Credit Amount,Client Reference,Additional Reference,Misc Referenc
0,18 Jul 2020,NaN,ADV,BILL,770.01,,DBSC-8888888884 : I-BANK,NaN,NaN
1,13 Jul 2020,NaN,GR,IBG,,43.57,CONCUR TECHNOLOGIES SG PTE LTD,C8888HFKX8888,SUPP
2,07 Jul 2020,NaN,INT,NaN,,36.00,Preferential rate based on total,monthly transactions,NaN
3,04 Jul 2020,NaN,ADV,BILL,2696.62,,ITX -S88888888 : I-BANK,NaN,NaN


In [35]:
mapping = {
    'Transaction Date': 'date',
    'Debit Amount': 'debit',
    'Credit Amount': 'credit',
    'Client Reference': 'payee_name'
}
date_format = '%d %b %Y'


In [36]:
# Select cols and rename
df = df[mapping.keys()]
data = df.rename(columns=mapping)
data['date'] = pd.to_datetime(data['date'], format=date_format)

In [37]:
data

,date,debit,credit,payee_name
0,2020-07-18,770.01,,DBSC-8888888884 : I-BANK
1,2020-07-13,,43.57,CONCUR TECHNOLOGIES SG PTE LTD
2,2020-07-07,,36.00,Preferential rate based on total
3,2020-07-04,2696.62,,ITX -S88888888 : I-BANK


In [8]:
from datetime import date

today = str(date.today()).replace('-','_')
print("Today's date:", today)

Today's date: 2020_07_25


In [38]:
data.to_csv('test.csv', index=False)